#  Préparation des Données de Crime pour PySpark

Ce notebook implémente un nettoyage ciblé pour optimiser le jeu de données en vue de son exploitation sur un cluster **PySpark**. L'accent est mis sur la **réduction dimensionnelle** et la **standardisation des types** (notamment les dates), facilitant les *transformations* et *actions* distribuées.

In [3]:
import pandas as pd
import numpy as np

# 1. Importation:
file = "Crime_Data_from_2020_to_Present.csv"
df = pd.read_csv(file)

## 1. Réduction Dimensionnelle (Suppression de Colonnes)

**Justification :** En PySpark, un DataFrame plus étroit (moins de colonnes) est essentiel pour minimiser les coûts de *shuffle* et de stockage. Nous conservons uniquement les colonnes strictement nécessaires à l'analyse en utilisant le QQOQCP (quoi, quand, où, qui/victime) et supprimons les colonnes redondantes (`Crm Cd 2/3/4`) ou trop dispersées (`Cross Street`, `LOCATION`, `DR_NO`).

In [5]:
colonnes_a_garder = [
    "DATE OCC", "TIME OCC", "AREA NAME", "Rpt Dist No",
    "Crm Cd", "Crm Cd Desc", "Vict Age", "Vict Sex", "Vict Descent",
    "Premis Cd", "Premis Desc", "Weapon Used Cd", "Weapon Desc",
    "Status", "LAT", "LON"]

df = df[[col for col in colonnes_a_garder if col in df.columns]].copy() # copie du DataFrame contenant les colonnes essentielles uniquement.
print(f"Le DataFrame a été réduit à {df.shape[1]} colonnes.")

Le DataFrame a été réduit à 16 colonnes.


In [6]:
# Mettre les colonnes en format snake case pour des raisons de convention:
df.columns= [col.lower().strip().replace(" ", "_") for col in df.columns]
df.columns 

Index(['date_occ', 'time_occ', 'area_name', 'rpt_dist_no', 'crm_cd',
       'crm_cd_desc', 'vict_age', 'vict_sex', 'vict_descent', 'premis_cd',
       'premis_desc', 'weapon_used_cd', 'weapon_desc', 'status', 'lat', 'lon'],
      dtype='object')

## 2. Traitement des Dates et Création de Caractéristiques Temporelles

**Justification :** PySpark manipule facilement les types `timestamp`. La combinaison des colonnes `date_occ` et `time_occ` en un `timestamp_occ` unique simplifiera les futures agrégations temporelles. Nous allons calculer des colonnes dérivées (`heure`, `jour_semaine`) pour accélérer les requêtes d'analyse.

In [8]:
# 1. Conversion de la colonne de DATE (chaîne de caractères):
date_format = "%Y %b %d %I:%M:%S %p" 
df["date_full"] = pd.to_datetime(df["date_occ"], format=date_format, errors="coerce")

# 2. Création du Timestamp complet:
# L'heure de l'événement est stockée en HHMM dans'time_occ'.On prend l'info de DATE de 'date_full' et prend le temps (HH:MM) de 'time_occ'.
df['time_occ'] = df['time_occ'].astype(str).str.zfill(4) # Assure le format 4 chiffres : 0000

# On prend la partie DATE de 'date_full', et on lui ajoute la partie TIME correcte de 'time_occ':
df['timestamp_occ'] = pd.to_datetime(\
    df['date_full'].dt.strftime('%Y-%m-%d') + ' ' + \
    df['time_occ'].str[:2] + ':' + df['time_occ'].str[2:] + ':00', # Ajout des secondes '00'
    errors='coerce'\
)
    
# 3. Suppression des colonnes initiales et intermédiaires:
df = df.drop(columns=["date_occ", "time_occ", "date_full"])

# 4. Création des colonnes dérivées:
df['heure'] = df['timestamp_occ'].dt.hour.astype('Int64')
df['jour_semaine'] = df['timestamp_occ'].dt.day_name()
    
print("Colonnes temporelles créées et nettoyées (timestamp_occ, heure, jour_semaine).")
display(df[['timestamp_occ', 'heure', 'jour_semaine']].head())

Colonnes temporelles créées et nettoyées (timestamp_occ, heure, jour_semaine).


,timestamp_occ,heure,jour_semaine
0,2020-11-07 08:45:00,8,Saturday
1,2020-10-18 18:45:00,18,Sunday
2,2020-10-30 12:40:00,12,Friday
3,2020-12-24 13:10:00,13,Thursday
4,2020-09-29 18:30:00,18,Tuesday


## 3. Traitement des Manquants (Imputation)

**Justification :** PySpark n'aime pas les `NULL` inattendus. Ils peuvent donner des résultats indésirables lors des agrégations et des jointures. Nous allons imputer les `NaN` par des valeurs comme la médiane pour les numériques, pour assurer la cohérence de données.

### A. 'vict_age' (MCAR) et Groupement

In [20]:
# Nettoyage des aberrants et conversion en NaN:
# Imputation des âges négatifs et >100 par "np.nan":
df["vict_age"] = pd.to_numeric(df["vict_age"], errors="coerce")
df.loc[(df["vict_age"] < 0) | (df["vict_age"] > 100), "vict_age"] = np.nan

# Imputation des manquants par la médiane:
median_age = df["vict_age"].median()
df["vict_age"] = df["vict_age"].fillna(median_age).astype("Int64")
    
# Groupement d'âge simplifié :
df["vict_age_group"] = pd.cut(
    df["vict_age"],
    bins=[0, 18, 35, 55, 100],
    labels=["Minor", "Young Adult", "Adult", "Senior"],
    include_lowest=True
)
print(f"Âge imputé (médiane) : {median_age}")

Âge imputé (médiane) : 30.0


### B. Variables Catégorielles (MNAR)

In [30]:
# vict_sex / descent : Remplacer par 'Unknown' :
df["vict_sex"] = df["vict_sex"].replace(["X", "H", "-", ""], np.nan).fillna("Unknown")
df["vict_descent"] = df["vict_descent"].fillna("Unknown")

# weapon_desc / code : Remplacer par 'No weapon' / 0 :
df["weapon_desc"] = df["weapon_desc"].fillna("No weapon")
df["weapon_used_cd"] = df["weapon_used_cd"].fillna(0).astype(int)

# premis_desc / code : Remplacer par 'NO DESC' / -1 (Manquants faibles, simple imputation)
df["premis_desc"] = df["premis_desc"].replace("-", np.nan).fillna("NO DESC")
df["premis_cd"] = df["premis_cd"].fillna(-1).astype(int)
    
# status : Remplacer par le mode (Manquant unique)
mode_status = df["status"].mode()[0]
df["status"] = df["status"].fillna(mode_status)

## 4. Vérification Finale

**Justification :** Une dernière vérification pour confirmer que les efforts de nettoyage ont réussi à éliminer les valeurs `NaN` sur les colonnes ciblées, garantissant une transition fluide vers PySpark.

In [80]:
print("\nTaille finale du jeu de données :  ", df.shape)
print("\nValeurs manquantes restantes    :\n")
display(df.isnull().sum())
print("\nAperçu du jeu de données final  :\n")
display(df.head())


Taille finale du jeu de données :   (1004991, 18)

Valeurs manquantes restantes    :



area_name         0
rpt_dist_no       0
crm_cd            0
crm_cd_desc       0
vict_age          0
vict_sex          0
vict_descent      0
premis_cd         0
premis_desc       0
weapon_used_cd    0
weapon_desc       0
status            0
lat               0
lon               0
timestamp_occ     0
heure             0
jour_semaine      0
vict_age_group    0
dtype: int64


Aperçu du jeu de données final  :



,area_name,rpt_dist_no,crm_cd,crm_cd_desc,vict_age,vict_sex,vict_descent,premis_cd,premis_desc,weapon_used_cd,weapon_desc,status,lat,lon,timestamp_occ,heure,jour_semaine,vict_age_group
0,N Hollywood,1502,354,THEFT OF IDENTITY,31,M,H,501,SINGLE FAMILY DWELLING,0,No weapon,IC,34.2124,-118.4092,2020-11-07 08:45:00,8,Saturday,Young Adult
1,N Hollywood,1521,230,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",32,M,H,102,SIDEWALK,200,KNIFE WITH BLADE 6INCHES OR LESS,IC,34.1993,-118.4203,2020-10-18 18:45:00,18,Sunday,Young Adult
2,Van Nuys,933,354,THEFT OF IDENTITY,30,M,W,501,SINGLE FAMILY DWELLING,0,No weapon,IC,34.1847,-118.4509,2020-10-30 12:40:00,12,Friday,Young Adult
3,Wilshire,782,331,THEFT FROM MOTOR VEHICLE - GRAND ($950.01 AND ...,47,F,A,101,STREET,0,No weapon,IC,34.0339,-118.3747,2020-12-24 13:10:00,13,Thursday,Adult
4,Pacific,1454,420,THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER),63,M,H,103,ALLEY,0,No weapon,IC,33.9813,-118.4350,2020-09-29 18:30:00,18,Tuesday,Senior
